<a href="https://colab.research.google.com/github/ales27pm/adastra/blob/main/monGARS_CPU_Colab_CLEAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# monGARS Fine-Tune (CPU/GPU compatible version)
Compatible avec Colab CPU ou GPU — utilise Transformers + PEFT (pas Unsloth).

In [1]:
# Install dependencies (HuggingFace Transformers + PEFT + Datasets)!pip install -q transformers peft datasets accelerate bitsandbytes

In [2]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
import torch
from datasets import interleave_datasets

In [3]:
# Load and prepare multilingual & Quebec French datasets
from datasets import load_dataset, interleave_datasets

def load_and_format(name, split, fn, limit=3000):
    ds = load_dataset(name, split=split).select(range(limit))
    return ds.map(fn, remove_columns=ds.column_names)

datasets = [
    load_and_format("squad", "train", lambda x: {
        "prompt": f"Q: {x['question']}\nContext: {x['context']}",
        "response": x['answers']['text'][0] if x['answers']['text'] else 'N/A'
    }),
    load_and_format("gsm8k", "train", lambda x: {
        "prompt": x['question'],
        "response": x['answer']
    }),
    load_and_format("MihaelaGaman/FreCDo", "train", lambda x: {
        "prompt": f"Utilisateur: {x['text']}",
        "response": "Bien noté."
    } if x.get("country") == "Canada" else None),
    load_and_format("GRAAL-Research/FCCLC", "train", lambda x: {
        "prompt": f"Utilisateur: {x['text']}",
        "response": "Voici une réponse adaptée..."
    })
]
datasets = [d.filter(lambda x: x['prompt'] and x['response']) for d in datasets]
dataset = interleave_datasets(datasets).train_test_split(test_size=0.1)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


ValueError: Invalid pattern: '**' can only be an entire path component

In [ ]:
# Load base model and tokenizermodel_id = 'tiiuae/falcon-rw-1b'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto')

In [ ]:
# Prepare PEFT with LoRApeft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)
model = get_peft_model(model, peft_config)

In [ ]:
# Tokenizationdef tokenize(example):
    text = example['prompt'] + '\n' + example['response']
    return tokenizer(text, truncation=True, max_length=512, padding='max_length')

train_dataset = dataset['train'].map(tokenize, remove_columns=dataset['train'].column_names)
eval_dataset = dataset['test'].map(tokenize, remove_columns=dataset['test'].column_names)

In [ ]:
# Training arguments and trainer setuptraining_args = TrainingArguments(
    output_dir="monGARS_finetuned_cpu",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,
    fp16=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)
trainer.train()